In [2]:
import xarray as xr
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

# === CONFIGURATION ===
base_dir = "/bsushare/leaf-shared/subset-output-wrf/vol04/wrf_out/wy_2000/d01/"
month = "03"
year = "2000"
target_location = "Boise"
target_lat = 43.615
target_lon = -116.202
epsilon = 1e-6  # to prevent division by zero

# === FUNCTION TO LOAD A VARIABLE ===
def load_wrf_var(var_name, base_dir, year, month):
    filename = f"{var_name}_{year}-{month}.nc"
    path = os.path.join(base_dir, filename)
    try:
        ds = xr.open_dataset(path)
        print(f"Loaded {filename}")
        return ds
    except FileNotFoundError:
        print(f"File not found: {filename}")
        return None

# === LOAD REQUIRED DATASETS ===
t2 = load_wrf_var("T2", base_dir, year, month)
snownc = load_wrf_var("SNOWNC", base_dir, year, month)
rainnc = load_wrf_var("RAINNC", base_dir, year, month)

# === FIND CLOSEST GRID CELL TO TARGET LOCATION ===
lats = t2['XLAT'][0, :, :].values
lons = t2['XLONG'][0, :, :].values
dist = np.sqrt((lats - target_lat)**2 + (lons - target_lon)**2)
i, j = np.unravel_index(np.argmin(dist), dist.shape)

print(f"Closest grid cell to {target_location}: i = {i}, j = {j}")
print(f"Coordinates: lat = {lats[i, j]:.3f}, lon = {lons[i, j]:.3f}")

# === EXTRACT TIME SERIES FOR TARGET LOCATION ===
temp_series = t2['T2'][:, i, j] - 273.15  # K to °C
snow_series = snownc['SNOWNC'][:, i, j]
rain_series = rainnc['RAINNC'][:, i, j]
time = t2['XTIME']
time_trimmed = time.isel(Time=slice(1, None))

# === DIFFERENCING FUNCTION TO HANDLE ROLLOVER ===
def compute_hourly_precip(accum_series):
    diff = accum_series.diff(dim="Time", label="upper")
    prev = accum_series[:-1]
    corrected = xr.where(diff < 0, diff + prev, diff)
    return corrected.clip(min=0)

# === HOURLY PRECIP TIME SERIES ===
snow_hourly = compute_hourly_precip(snow_series)
rain_hourly = compute_hourly_precip(rain_series)
temp_series_trimmed = temp_series.isel(Time=slice(1, None))

# === FRACTION CALCULATIONS ===
snow_frac = snow_hourly / (rain_hourly + epsilon)

# === COMBINE DATASET ===
combined = xr.Dataset({
    "temp_C": temp_series_trimmed,
    "snow_mm": snow_hourly,
    "rain_mm": rain_hourly,
    "snow_frac": snow_frac
}, coords={"time": time_trimmed})

# === PLOT TEMPERATURE + PRECIP ===
plt.figure(figsize=(10, 4))
plt.plot(time_trimmed, combined['temp_C'], label="Temp (°C)")
plt.plot(time_trimmed, combined['snow_mm'], label="Snowfall (mm)")
plt.plot(time_trimmed, combined['rain_mm'], label="Rain (mm)")
plt.legend()
plt.title(f"Weather at {target_location}, {month}-{year}")
plt.xlabel("Time")
plt.ylabel("Value")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# === PLOT FRACTIONS ===
plt.figure(figsize=(10, 3))
plt.plot(time_trimmed, combined['snow_frac'], label="Snow / Rain")
plt.legend()
plt.title(f"Snow Fraction at {target_location}, {month}-{year}")
plt.xlabel("Time")
plt.ylabel("Fraction")
plt.ylim(0, 2)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# === PLOT RAIN AND SNOW SIDE BY SIDE ===
fig, axs = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

axs[0].plot(time_trimmed, combined['rain_mm'], label="Rainfall (mm)", color="green")
axs[0].set_ylabel("Rain (mm)")
axs[0].legend()

axs[1].plot(time_trimmed, combined['snow_mm'], label="Snowfall (mm)", color="blue")
axs[1].set_ylabel("Snow (mm)")
axs[1].legend()

plt.xlabel("Time")
plt.xticks(rotation=45)
plt.suptitle(f"Rain and Snow at {target_location}, {month}-{year}")
plt.tight_layout()
plt.show()

# FUTURE:
# === LOGISTIC REGRESSION: Liquid (1) vs Solid (0) Precip ===
# precip_mask = (combined['rain_mm'] + combined['snow_mm']) > 0
# solid_mask = combined['snow_mm'] > combined['rain_mm']
# precip_type = xr.where(solid_mask, 0, 1)

# df = pd.DataFrame({
#     "temp_C": combined["temp_C"].where(precip_mask).values,
#     "precip_type": precip_type.where(precip_mask).values
# }).dropna()

# X = sm.add_constant(df["temp_C"])
# y = df["precip_type"]

# model = sm.Logit(y, X).fit()
# print(model.summary())

File not found: T2_2000-03.nc
File not found: SNOWNC_2000-03.nc
File not found: RAINNC_2000-03.nc


TypeError: 'NoneType' object is not subscriptable